In [2]:
pip install pdfplumber

In [3]:
import spacy

# Load spaCy's English NLP model
nlp = spacy.load("en_core_web_sm")

In [4]:
import pdfplumber
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [38]:
import pdfplumber

cleaned_text = ""  # Initialize an empty string to accumulate all the cleaned text

with pdfplumber.open('practice-standard-project-risk-management.pdf') as pdf:
    for page in pdf.pages[12:]:
        text = page.extract_text()
        if text:
            footer_text = "©2009 Project Management Institute. Practice Standard for Project Risk Management"
            # Remove footer and lines containing 'Figure'
            cleaned_page_text = "\n".join([line for line in text.split('\n') if footer_text not in line and 'Figure' not in line])
            cleaned_text += cleaned_page_text + "\n"  # Accumulate the cleaned text

In [39]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [40]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [41]:
print(cleaned_text[:500])

1
CHAPTER 1
INTRODUCTION
P roject Management Institute (PMI) practice standards are guides to the use of a tool, technique, or process
identifi ed in A Guide to the Project Management Body of Knowledge ( PMBOK ® G uide – Fourth Edition) or
other PMI standards. Practice standards are targeted at audiences who participate in the management of
projects. This includes project managers, project personnel, contract personnel, supervisors, and other project
stakeholders.
A PMI practice standard describ


In [42]:
doc = nlp(cleaned_text)

print(doc)

1
CHAPTER 1
INTRODUCTION
P roject Management Institute (PMI) practice standards are guides to the use of a tool, technique, or process
identifi ed in A Guide to the Project Management Body of Knowledge ( PMBOK ® G uide – Fourth Edition) or
other PMI standards. Practice standards are targeted at audiences who participate in the management of
projects. This includes project managers, project personnel, contract personnel, supervisors, and other project
stakeholders.
A PMI practice standard describes processes, activities, inputs, and outputs for a specifi c Knowledge Area.
It provides information on what the signifi cant process, tool, or technique is, what it does, why it is signifi cant,
when it should be performed or executed, and, if necessary for further clarifi cation, who should perform the
process. A practice standard does not prescribe how the process is to be implemented, leaving that subject for
other forums such as handbooks, manuals, and courses.
This chapter includes the fo

In [43]:
token_details = []

for idx, token in enumerate(doc):
    if not token.is_space and not token.is_punct:
        token_details.append([idx, token.text, token.lemma_, token.pos_, token.tag_, token.dep_])

In [44]:
from tabulate import tabulate
print(tabulate(token_details[:25],headers=['ID','TEXT','LEMMA','POS','TAG','DEP']))

  ID  TEXT          LEMMA         POS    TAG    DEP
----  ------------  ------------  -----  -----  --------
   0  1             1             NUM    CD     nummod
   2  CHAPTER       chapter       NOUN   NN     nmod
   3  1             1             NUM    CD     nummod
   5  INTRODUCTION  introduction  NOUN   NN     compound
   7  P             p             NOUN   NN     nsubj
   8  roject        roject        VERB   VBP    ROOT
   9  Management    Management    PROPN  NNP    compound
  10  Institute     Institute     PROPN  NNP    nmod
  12  PMI           PMI           PROPN  NNP    appos
  14  practice      practice      NOUN   NN     compound
  15  standards     standard      NOUN   NNS    nsubj
  16  are           be            AUX    VBP    ccomp
  17  guides        guide         NOUN   NNS    acomp
  18  to            to            ADP    IN     prep
  19  the           the           DET    DT     det
  20  use           use           NOUN   NN     pobj
  21  of            of 

In [45]:
from spacy import displacy
sentences = list(doc.sents)[:100]
for sent in sentences:
  displacy.render(sent,style='dep',jupyter=True,options={'distance':100})

In [46]:
ner_details=[]
for ent in doc.ents:
  ner_details.append([ent.text,ent.start_char,ent.end_char,ent.label_])

In [47]:
import pandas as pd
df=pd.DataFrame(ner_details,columns=['TEXT','START','END','LABEL'])
df

,TEXT,START,END,LABEL
0,1,0,1,CARDINAL
1,CHAPTER 1,2,11,LAW
2,P roject Management Institute,25,54,ORG
3,PMI,56,59,ORG
4,the Project Management Body of Knowledge,161,201,ORG
...,...,...,...,...
2063,18,201084,201086,DATE
2064,43,201088,201090,DATE
2065,46,201092,201094,DATE
2066,48,201096,201098,CARDINAL


In [48]:
displacy.render(sentences,style='ent',jupyter=True)

/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [49]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\[.*?\]', '', text)  # Remove references
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with digits
    text = re.sub('http\S+\s', ' ', text)

    # Tokenize, lemmatize, and remove stop words using spaCy
    doc = nlp(text)

    # Return lemmatized tokens that are not stop words or punctuations
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return tokens

In [50]:
# Step 2: Preprocess the text (tokenization, normalization, lemmatization)
cleaned_tokens = preprocess_text(cleaned_text)
cleaned_text_rejoined = ' '.join(cleaned_tokens)

In [51]:
doc_cleaned = nlp(cleaned_text_rejoined)

In [52]:
for chunk in doc_cleaned.noun_chunks:
    chunk_text = chunk.text.strip()  # Remove leading and trailing spaces
    if chunk_text:  # Check if the chunk is not empty
        print(f"Noun Chunk: {chunk_text.capitalize()}")

Noun Chunk: Chapter 
 introduction
Noun Chunk: P roject management institute pmi practice
Noun Chunk: Standard guide
Noun Chunk: Tool technique process
Noun Chunk: Project management body knowledge
Noun Chunk: Project manager project personnel contract personnel supervisor project 
 stakeholder 
 pmi practice standard describe process activity input output specifi c knowledge area
Noun Chunk: Information signifi
Noun Chunk: Tool technique signifi
Noun Chunk: Necessary clarifi cation perform 
 process practice standard prescribe process
Noun Chunk: Leave subject 
 forum
Noun Chunk: Good risk management practice
Noun Chunk: Standard project 
 management practitioner stakeholder
Noun Chunk: Ne aspect project risk management
Noun Chunk: Good practice project time b
Noun Chunk: Standard
Noun Chunk: Educational purpose
Noun Chunk: Single 
 project
Noun Chunk: P mbok
Noun Chunk: Fourth edition practice standard cover risk program 
 portfolio project
Noun Chunk: C hapter   p mbok
Noun Chunk: F

In [53]:
for token in doc_cleaned:
    if token.pos_ == "VERB":
        verb_phrase = " ".join([token.text] + [child.text for child in token.children])
        print(f"Verb Phrase: {verb_phrase}")

Verb Phrase: use introduction guide process guide
Verb Phrase: include project provide
Verb Phrase: provide area signifi process perform implement include apply guide include
Verb Phrase: process not signifi
Verb Phrase: perform not execute
Verb Phrase: execute process
Verb Phrase: perform cation 
 
Verb Phrase: implement
Verb Phrase: handbook forum
Verb Phrase: include chapter provide
Verb Phrase: provide management stakeholder defi recognize
Verb Phrase: recognize management provide
Verb Phrase: provide b standard applicable
Verb Phrase: apply consistently apply
Verb Phrase: apply management project guide
Verb Phrase: guide like mbok    project
Verb Phrase: guide    emphasize
Verb Phrase: emphasize chapter textbook
Verb Phrase: relate
Verb Phrase: organize standard section
Verb Phrase: include material defi
Verb Phrase: defi process guide
Verb Phrase: guide pmbok    g
Verb Phrase: follow process 
  
Verb Phrase: identify follow management risk perform perform describe
Verb Phrase: pe

In [55]:
pos_tags = [(token.text, token.pos_) for token in doc_cleaned if not token.is_punct and not token.is_space]
print(pos_tags[:20])

[('chapter', 'NOUN'), ('introduction', 'NOUN'), ('p', 'PROPN'), ('roject', 'PROPN'), ('management', 'PROPN'), ('institute', 'PROPN'), ('pmi', 'PROPN'), ('practice', 'PROPN'), ('standard', 'ADJ'), ('guide', 'NOUN'), ('use', 'VERB'), ('tool', 'NOUN'), ('technique', 'NOUN'), ('process', 'NOUN'), ('identifi', 'PROPN'), ('ed', 'PROPN'), ('guide', 'PROPN'), ('project', 'NOUN'), ('management', 'NOUN'), ('body', 'NOUN')]


In [57]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF to find the most important terms
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([cleaned_text_rejoined])
feature_names = tfidf_vectorizer.get_feature_names_out()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
tfidf_results = list(zip(feature_names, denselist[0]))

# Sort by TF-IDF score
tfidf_results = sorted(tfidf_results, key=lambda x: x[1], reverse=True)

for term, score in tfidf_results[:50]:  # Show top 10 results
    print(term + ': ' + str(score))

print("Processing complete.")


risk: 0.7469693035654834
project: 0.44899794203389715
management: 0.24694886811864342
process: 0.12908690833474543
analysis: 0.11531083511325085
response: 0.11224948550847429
plan: 0.10459611149653285
pmp: 0.09490183774807372
objective: 0.07908486479006142
technique: 0.06990081597573171
identifi: 0.06837014117334343
action: 0.05918609235901371
chapter: 0.05357361808359
stakeholder: 0.05255316821533114
identify: 0.04796114380816628
require: 0.04694069393990743
impact: 0.04592024407164857
level: 0.04592024407164857
standard: 0.04592024407164857
cost: 0.043369119401001424
quantitative: 0.042858894466872
ed: 0.04234866953274257
include: 0.04234866953274257
provide: 0.04183844459861314
cation: 0.04132821966448371
defi: 0.04132821966448371
information: 0.04030776979622486
overall: 0.038777094993836574
perform: 0.03724642019144828
probability: 0.03673619525731885
tool: 0.03673619525731885
example: 0.03571574538906
result: 0.03571574538906
effect: 0.03520552045493057
practice: 0.03520552045493

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
